# Úvod do web scrapingu s BeautifulSoup

**BeautifulSoup** je Python knihovna pro web scraping. Sama o sobě nestahuje žádná data - pouze je zpracovává. Data musíme získat pomocí knihovny `requests`.

BeautifulSoup je knihovna pro práci s XML, přizpůsobená pro parsování webových stránek.

## Import knihoven

Knihovna BeautifulSoup se nachází v modulu `bs4`.

In [1]:
import requests
from bs4 import BeautifulSoup

## Základní použití BeautifulSoup

Nejprve stáhneme obsah webové stránky pomocí `requests.get()`, poté předáme text stránky konstruktoru `BeautifulSoup`.

In [ ]:
# Velmi zjednoduseny kod stranky
minit = """<!doctype html>
<html>
  <head>
    <title>Haiku</title>
    <meta charset="UTF-8">
  </head>
  <body>
    <div id="haiku" class="tiny">
     At a fishmonger's<br>
     Brackish seawater breams' gums<br>
     Are icily cold.
     <ul>
         <li>the end</li>
         <li><a href = "http://wiki.org">Wiki</li>
     <ul>
    </div>
    <div id="something">
        <ul class="tiny">
            <li>Something</li>
            <li>Other</li>
            <li>Written.</li>
        </ul>
    </div>

    <footer>CodersLab</footer>
  </body>
</html>"""

minis = BeautifulSoup(minit, 'html.parser')

Parametr `'html.parser'` říká, že budeme parsovat HTML kód. Existují i jiné parsery (např. `xml` pro XML soubory).

Nyní máme objekt `soup`, který obsahuje data celé stránky.

## Metody pro extrakci dat

Pro vyhledávání elementů používáme čtyři hlavní metody:

- `.find()` - vrátí jeden element odpovídající vzoru
- `.find_all()` - vrátí seznam všech odpovídajících elementů
- `.select_one()` - vrátí jeden element podle CSS selektoru
- `.select()` - vrátí seznam elementů podle CSS selektoru

### Vyhledávání podle tagu

Nejjednodušší způsob - hledáme element podle názvu HTML tagu.

In [ ]:
minis.find('div')

In [ ]:
minis.find('br')

In [ ]:
minis.select_one('div')

Pokud metody `.find()` nebo `.select_one()` nenajdou žádný element, vrátí `None`.

### Vyhledávání podle id

Atribut `id` by měl být na stránce unikátní.

In [ ]:
# Pomocí metody find() s parametrem id
minis.find(id='something')

In [ ]:
# Pomocí CSS selektoru - znak # označuje id
minis.select_one('#something')

### Vyhledávání podle třídy (class)

Pozor: v Pythonu je `class` rezervované slovo, proto používáme `class_` s podtržítkem.

In [ ]:
# Pomocí metody find() s parametrem class_
minis.find(class_ = 'poem_short')

In [ ]:
# Pomocí CSS selektoru - tečka označuje třídu
minis.select_one('.poem_short')

### Kombinace více parametrů

Můžeme kombinovat tag, id a třídu pro přesnější vyhledávání.

In [ ]:
# Hledání tlačítka s konkrétním id
minis.find('ul', class_= 'tiny')

In [ ]:
# Stejný výsledek pomocí CSS selektoru
minis.select_one('ul.tiny')

### Vyhledávání v nalezeném elementu

HTML má stromovou strukturu. Můžeme nejprve najít nadřazený element a pak v něm hledat.

In [ ]:
# Nejprve vybereme "article"
article = minis.find('div')

In [ ]:
# Pak v něm hledáme položky seznamu
if article:
    print(article.find('li'))

### Získání více elementů najednou

Metody `.find_all()` a `.select()` vrací seznam všech nalezených elementů.

Pokud `.find_all()` nebo `.select()` nic nenajdou, vrátí prázdný seznam `[]`.

In [ ]:
# Najděte všechny li elementy a vypište je


In [ ]:
# Vypište jenom jejich text


In [ ]:
for li in minis.find_all('li'):
    print(li)

In [ ]:
[li for li in minis.find_all('li')]

In [ ]:
[li.get_text() for li in minis.find_all('li')]

In [ ]:
', '.join([li.get_text() for li in minis.find_all('li')])

### Úloha

In [ ]:
# Hledání prvního elementu s tagem <div>


In [ ]:
# Najděte element s id = 'cookie-banner'


In [ ]:
# Najděte první element třídy 'swiper-slide'


Element může mít více tříd oddělených mezerou: `<span class="btn btn-primary">`. 
V CSS selektoru stačí uvést jednu, nebo můžeme obě: `.btn.btn-primary`

In [ ]:
# Najděte první element, který má obě třídy 'btn-primary' i 'btn'


In [ ]:
# Najděte tlačítko (ne libovolný element) s id 'reject-cookies'


In [ ]:
# Najděte element s id 'cookie-banner' Pak v tom elementu najděte tlačítko s id 'cookie-preferences'



In [ ]:
# Najděte všechny <span> elementy z webu, zobrazte jenom prvních 5. Kolik jich je?


## Vlastnosti nalezeného elementu

Metody `.find()`, `.select_one()`, `.select()` a `.find_all()` vrací objekt (nebo více objektů) reprezentující element webové stránky. Tento objekt má několik užitečných vlastností:

- `.name` - název tagu
- `[...]` - hodnota atributu tagu (např. `link['href']` - adresa, na kterou odkaz vede)
- `.attrs` - atributy jako slovník
- `.text` - textový obsah tagu

In [ ]:
minis

In [ ]:
# Najdeme první div


In [ ]:
# Název tagu


In [ ]:
# Textový obsah


In [ ]:
# Všechny atributy jako slovník


### Získání hodnoty odkazu
U odkazů často potřebujeme získat atribut `href`.

In [ ]:
# Najdeme první odkaz
link = minis.find('a')

if link:
    print("Text odkazu:", link.text)
    print("URL:", link['href'])

### Formátovaný výpis - prettify()

Metoda `.prettify()` vrátí HTML kód s pěkným odsazením pro lepší čitelnost.

In [ ]:
print(result)

In [ ]:
print(result.prettify())

---

## Praktický příklad: Top 15 skladeb

Na stránce music-to-scrape.org chceme získat aktuální top 15 skladeb a vypsat je ve formátu "Artist: {artist} | track: {track}".

Postup:
1. Najdeme kontejner se seznamem skladeb
2. Najdeme jednotlivé skladby
3. Z každé skladby vytáhneme interpreta a název

In [ ]:
#import requests
#from bs4 import BeautifulSoup

# Stáhneme stránku
r = requests.get("https://www.music-to-scrape.org/")
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
# Najdeme sekci s name = 'weekly_15'


In [ ]:
# Najdeme všechny skladby v té sekci
tracks = 

In [ ]:
print(f"Nalezeno {len(tracks)} skladeb")

In [ ]:
# Projdeme všechny skladby a vypíšeme informace
for track in tracks:


---

## Otázky k zamyšlení

**Otázka 1:** Jaký je rozdíl mezi `.find()` a `.find_all()`?

In [ ]:
# Vaše odpověď:
# 

**Otázka 2:** Co vrátí `.find()`, pokud element nenajde? A co vrátí `.find_all()`?

In [ ]:
# Vaše odpověď:
# 

**Otázka 3:** Proč používáme `class_` místo `class` v metodě `.find()`?

In [ ]:
# Vaše odpověď:
# 

---

## Oprav chyby v kódu

**Úloha 1:** Následující kód obsahuje chybu. Opravte ji.

In [ ]:
# Chybný kód - opravte
from bs4 import BeautifulSoup
import requests

r = requests.get("https://www.music-to-scrape.org/")
soup = BeautifulSoup(r.text)  

**Úloha 2:** Následující kód hledá element podle třídy, ale obsahuje chybu. Opravte ji.

In [ ]:
# Chybný kód - opravte
result = soup.find(class='swiper-slide') 

**Úloha 3:** Tento kód má vracet text elementu, ale nefunguje správně. Co je špatně?

In [ ]:
# Chybný kód - opravte
result = soup.find_all('p')
print(result)  

---

## Úloha: Prozkoumejte webovou stránku

Otevřete stránku [https://quotes.toscrape.com/](https://quotes.toscrape.com/) a pomocí nástrojů pro vývojáře (F12 nebo pravé tlačítko myši -> Prozkoumat element) najděte:

1. Element, který obsahuje první citát na stránce
2. Napište CSS selektor pro nalezení textu citátu
3. Napište CSS selektor pro nalezení autora citátu

In [ ]:
# import requests
# from bs4 import BeautifulSoup

# Vaše řešení zde:
# ..

# Najděte první citát a vypište jeho text a autora
# ...


---

## Úloha: Stáhněte všechny citáty

Na stránce [https://quotes.toscrape.com/](https://quotes.toscrape.com/) najděte všechny citáty a vypište je ve formátu:

```
"Text citátu" - Autor
```

In [ ]:
# import requests
# from bs4 import BeautifulSoup

# Vaše řešení zde:
# ..

# Najděte všechny citáty a vypište je


---

## Přehled použitých metod a funkcí

| Metoda/Funkce | Popis |
|--------------|-------|
| `requests.get(url)` | Stáhne obsah webové stránky |
| `BeautifulSoup(text, 'html.parser')` | Vytvoří objekt pro parsování HTML |
| `.find(tag, id=, class_=)` | Najde první element odpovídající kritériím |
| `.find_all(tag, id=, class_=)` | Najde všechny elementy odpovídající kritériím |
| `.select_one(css_selector)` | Najde první element podle CSS selektoru |
| `.select(css_selector)` | Najde všechny elementy podle CSS selektoru |
| `.text` | Vrátí textový obsah elementu |
| `.name` | Vrátí název HTML tagu |
| `.attrs` | Vrátí slovník všech atributů elementu |
| `element['atribut']` | Vrátí hodnotu konkrétního atributu |
| `.prettify()` | Vrátí formátovaný HTML kód pro lepší čitelnost |

### CSS selektory - rychlý přehled

| Selektor | Význam |
|----------|--------|
| `div` | Element s tagem div |
| `#id` | Element s daným id |
| `.trida` | Element s danou třídou |
| `.trida1.trida2` | Element s oběma třídami |
| `[name='hodnota']` | Element s atributem name="hodnota" |

---
## Dokumentace

BeautifulSoup má velmi dobrou dokumentaci dostupnou na:
https://www.crummy.com/software/BeautifulSoup/bs4/doc/